### Exemplo de Uso da API de stock market

Primeiro, o usuário deve se registrar no banco de dados, fornecendo username, password e uma confirmação do password

In [1]:
# Registrando no banco de dados        
import requests
from datetime import datetime
import pandas as pd
import time
import json

urlbase = 'https://stock-market-fgv.herokuapp.com/'

url = urlbase + 'register'
auth_dict = {'username': 'User Test Heroku 4', 'password':'teste'}

payload = auth_dict.copy()
payload['confirmation'] = 'teste'
x = requests.post(url, data = payload)
print(x.text)

Registered, user id: 43


#### Consultando Status

A rota ***/status*** fornece informações sobre o estado atual da carteira

In [2]:

# Verificando o status
url = urlbase + 'status'
x = requests.post(url, data = auth_dict)
print(x.text)

{"cash":10000,"shares":0,"symbol":"IFGV11","username":"User Test Heroku 4"}



#### Para consultar cotação atual

Usar a rota ***\quotations***, passando, além dos dados de autenticação (usuário e senha), o símbolo do ativo

In [8]:
# Consultando cotação

url = urlbase + 'quotations'
payload = auth_dict.copy()
payload['symbol'] = 'IFGV11'
x = requests.post(url, data = payload)
print(x.text)

{"time":"2021-03-21 19:26:00","price":118.904875072,"ticker":"IFGV11"}


#### Para cotação histórica

In [9]:
url = urlbase + 'past_quotations'

payload = auth_dict.copy()
payload['symbol'] = 'IFGV11'
payload['date'] = '2020-12-22'
x = requests.post(url, data = payload)
print(x.text)

{"time":{"0":"2020-12-22 17:00:00","1":"2020-12-22 17:01:00","2":"2020-12-22 17:02:00","3":"2020-12-22 17:03:00","4":"2020-12-22 17:04:00","5":"2020-12-22 17:05:00","6":"2020-12-22 17:06:00","7":"2020-12-22 17:07:00","8":"2020-12-22 17:08:00","9":"2020-12-22 17:09:00","10":"2020-12-22 17:10:00","11":"2020-12-22 17:11:00","12":"2020-12-22 17:12:00","13":"2020-12-22 17:13:00","14":"2020-12-22 17:14:00","15":"2020-12-22 17:15:00","16":"2020-12-22 17:16:00","17":"2020-12-22 17:17:00","18":"2020-12-22 17:18:00","19":"2020-12-22 17:19:00","20":"2020-12-22 17:20:00","21":"2020-12-22 17:21:00","22":"2020-12-22 17:22:00","23":"2020-12-22 17:23:00","24":"2020-12-22 17:24:00","25":"2020-12-22 17:25:00","26":"2020-12-22 17:26:00","27":"2020-12-22 17:27:00","28":"2020-12-22 17:28:00","29":"2020-12-22 17:29:00","30":"2020-12-22 17:30:00","31":"2020-12-22 17:31:00","32":"2020-12-22 17:32:00","33":"2020-12-22 17:33:00","34":"2020-12-22 17:34:00","35":"2020-12-22 17:35:00","36":"2020-12-22 17:36:00","3

#### Para compra

Usar a rota ***\buy***, passando, além dos dados de autenticação (usuário e senha), o símbolo do ativo a ser adquirido e a quantidade.

In [10]:
# Comprando

url = urlbase + 'buy'
payload = auth_dict.copy()
payload['symbol'] = 'IFGV11'
payload['shares'] = 20
x = requests.post(url, data = payload)
print(x.text)

Compra realizada com sucesso: 20 IFGV11 por 118.904875072 @ 19:26:31 21/03/2021


In [11]:
# Verificando o status
url = urlbase + 'status'
x = requests.post(url, data = auth_dict)
print(x.text)

{"cash":7621.90249856,"shares":20,"symbol":"IFGV11","username":"User Test Heroku"}



#### Consultando histórico

A rota ***/history*** fornece informações sobre o histórico de transações do usuário. Como acabamos de cadastrar, o histórico está vazio.

In [12]:
# Verificando o histórico
url = urlbase + 'history'
x = requests.post(url, data = auth_dict)
print(x.text)

{"Price":["118.904875072"],"Shares":[20],"Symbol":["IFGV11"],"transacted":["19:26:31 21/03/2021"]}



#### Para venda

Utilizar a rota ***/sell*** para vender ativos. Informar o símbolo do ativo e a quantidade a ser vendida

In [13]:
# Vendendo

url = urlbase + 'sell'
payload = auth_dict.copy()
payload['symbol'] = 'IFGV11'
payload['shares'] = 10
x = requests.post(url, data = payload)
print(x.text)

Venda realizada com sucesso: 10 IFGV11 por 118.904875072 @ 19:26:48 21/03/2021


Caso a quantidade a ser vendida não esteja disponível, o usuário recebe um '400' q a mensagem que os ativos não estão disponíveis

In [14]:
# Verificando o status
url = urlbase + 'status'
x = requests.post(url, data = auth_dict)
print(x.text)

{"cash":8810.95124928,"shares":10,"symbol":"IFGV11","username":"User Test Heroku"}



## Exemplo de Operação Simples
Apenas um exemplo utilizando médias móveis para decidir se compra ou vende o ativo.


In [15]:
# Consultando cotação

def get_quot(auth_dict, urlbase):
    import json
    url = urlbase + 'quotations'

    payload = auth_dict.copy()
    payload['symbol'] = 'IFGV11'

    x = requests.post(url, data = payload)

    dict_quot = json.loads(x.text)
    
    return dict_quot['price']

# Vendendo
def sell(qtdade,auth_dict, urlbase):
    url = urlbase + 'sell'

    payload = auth_dict.copy()
    payload['symbol'] = 'IFGV11'
    payload['shares'] = qtdade

    x = requests.post(url, data = payload)
    print(x.text)

# Comprando
def buy(qtdade,auth_dict, urlbase):
    url = urlbase + 'buy'

    payload = auth_dict.copy()
    payload['symbol'] = 'IFGV11'
    payload['shares'] = qtdade

    x = requests.post(url, data = payload)
    print(x.text)
    
def check_wallet(auth_dict, urlbase):   
    # Verificando o status
    url = urlbase + 'status'
    x = requests.post(url, data = auth_dict)
    dict_status = json.loads(x.text)

    return dict_status['cash'], dict_status['shares']

In [16]:
# Nesta célula, vamos apenas preencher os vetores com as últimas 5 e 10 últimas cotações do ativo
last_5_prices = pd.Series()
last_10_prices = pd.Series()

# preenche os vetores com historico recente:
for ii in range(0,5):    
    this_price = get_quot(auth_dict, urlbase)
    last_5_prices = last_5_prices.append(pd.Series([this_price]))
    time.sleep(60)
        
last_5_prices.reset_index(drop = True, inplace = True)
last_10_prices = last_5_prices.copy()

for ii in range(0,5):
    this_price = get_quot(auth_dict, urlbase)
    last_10_prices = last_10_prices.append(pd.Series([this_price]))
    time.sleep(60)
last_10_prices.reset_index(drop = True, inplace = True)


C:\Users\Borba\anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
C:\Users\Borba\anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  This is separate from the ipykernel package so we can avoid doing imports until


In [17]:
# Logica Simples: sempre que a media movel curta estiver acima da longa, compra. Sempre que tiver abaixo, vende.
qtade = 1
now = datetime.now()
current_time = now.strftime("%H:%M:%S")

while current_time < '17:00:00':

    # Remove o preço mais antigo dos vetores
    last_10_prices.pop(last_10_prices.index.min())
    last_5_prices.pop(last_5_prices.index.min())
    
    # Pega a cotação atual
    this_price = get_quot(auth_dict, urlbase)
    
    # Adiciona o preço mais atual nos vetores
    last_5_prices[last_5_prices.index.max()+1] = this_price
    last_10_prices[last_10_prices.index.max()+1] = this_price
    
    # Calcula as médias
    short_ave = last_5_prices.mean()
    long_ave = last_10_prices.mean()
    
    cash, shares = check_wallet(auth_dict, urlbase)
    
    if short_ave > 1.0001 * long_ave:
        # compra 1 se a media curta for maior que a longa
        if cash >= qtade*this_price:
            # Possui dinheiro para a compra
            buy(1,auth_dict, urlbase)
            print("Compra")
        else:
            print("Sem dinheiro para compra")
            
            
    elif short_ave <0.9999 * long_ave:
        # vende 1 se a media curta for menor que a longa
        if shares >= qtade:
            # Possui unidades para a venda
            sell(1,auth_dict, urlbase)
            print("Venda")
        else:
            print("Sem unidades para venda")
        
    else:
        print('Sem compra/ Venda')
    
    cash, shares = check_wallet(auth_dict, urlbase)
    patrimonio_total = round(cash + shares*this_price,2)
    print("Patrimônio total estimado: R$" + str(patrimonio_total))
    time.sleep(60)
    current_time = datetime.now().strftime("%H:%M:%S")

Compra realizada com sucesso: 1 IFGV11 por 118.382518395 @ 19:37:21 21/03/2021
Compra
Patrimônio total estimado: R$9994.78
Compra realizada com sucesso: 1 IFGV11 por 118.382518395 @ 19:38:28 21/03/2021
Compra
Patrimônio total estimado: R$9994.78
Compra realizada com sucesso: 1 IFGV11 por 118.400846699 @ 19:39:33 21/03/2021
Compra
Patrimônio total estimado: R$9995.0
Sem compra/ Venda
Patrimônio total estimado: R$9994.76
Venda realizada com sucesso: 1 IFGV11 por 118.446667461 @ 19:41:44 21/03/2021
Venda
Patrimônio total estimado: R$9995.59
Sem compra/ Venda
Patrimônio total estimado: R$9995.59
Compra realizada com sucesso: 1 IFGV11 por 118.428339156 @ 19:43:53 21/03/2021
Compra
Patrimônio total estimado: R$9995.37
Venda realizada com sucesso: 1 IFGV11 por 117.786848501 @ 19:44:58 21/03/2021
Venda
Patrimônio total estimado: R$9987.03
Venda realizada com sucesso: 1 IFGV11 por 117.548580543 @ 19:46:04 21/03/2021
Venda
Patrimônio total estimado: R$9984.17
Venda realizada com sucesso: 1 IFGV1